# NOTE

This notebook is an earlier version of the (Example_spec_redshifting)[Example_spec_redshifting.ipynb] notebook, but with the notebook gui code integrated in the notebook.  It is meant to demonstrate an earlier step in the development process, not the "final" notebook.

In [ ]:
import io

import numpy as np

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy import table

import specutils, specutils.fitting, specutils.analysis

from astroquery.sdss import SDSS

from IPython import display

%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
spec_file = SDSS.get_spectra(plate=6497, mjd=56329, fiberID=139)[0]
spec_file.writeto('spectrum.fits', overwrite=True)
spec = specutils.Spectrum1D.read('spectrum.fits', format='SDSS-III/IV spec')

In [ ]:
url = 'http://skyserver.sdss.org/dr12/SkyserverWS/ImgCutout/getjpeg?ra={}&dec={}&width=256&height=256&scale=.1'.format(spec_file[0].header['PLUG_RA'], spec_file[0].header['PLUG_DEC'])
display.Image(url=url)

Create a continuum-subtracted spectrum:

In [ ]:
basic_continuum = specutils.fitting.fit_generic_continuum(spec)
subspec = spec - basic_continuum(spec.spectral_axis)

Define some important known bright galaxy emission lines:

In [ ]:
bright_lines = {'Ha': 6562.801*u.AA, 'Hb':4861.363*u.AA, 'OIIIa':4958.911*u.AA, 'OIIIb':5006.843*u.AA,
                'OII':3727*u.AA # a doublet so this is ballpark
               } 

# Identify line regions 

## Non-interactive version

In [ ]:
plt.step(subspec.wavelength.to(u.angstrom), subspec.flux)

That looks like a classic emission line spectrum, but now I need to zoom in by hand to find the lines.  Lets start with $H\alpha$

In [ ]:
plt.step(subspec.wavelength.to(u.angstrom), subspec.flux)
plt.xlim(7000, 7200)

In [ ]:
plt.step(subspec.wavelength, subspec.flux, where='mid')
plt.xlim(7100, 7150)

halpha_region = specutils.SpectralRegion(7120*u.angstrom, 7135*u.angstrom)
plt.axvline(halpha_region.lower.to(subspec.wavelength.unit).value, c='k', ls='--')
plt.axvline(halpha_region.upper.to(subspec.wavelength.unit).value, c='k', ls='--')

halpha_center = specutils.analysis.centroid(subspec, halpha_region)
plt.axvline(halpha_center.value, c='k', ls='-')

In the above I had to hand-adjust the locations of the lines to find the right wl bounds.  So now I can *guess* the redshift:

In [ ]:
halpha_z = halpha_center/bright_lines['Ha'] - 1
halpha_z

Which lets me estimate where all the other lines should be:

In [ ]:
for nm, wl in bright_lines.items():
    print(nm, (1+halpha_z)*wl)

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 6))

regions = {'Ha': halpha_region,
           'Hb': specutils.SpectralRegion(5275*u.angstrom, 5285*u.angstrom),
           'OIIIa': specutils.SpectralRegion(5380*u.angstrom, 5390*u.angstrom),
           'OIIIb': specutils.SpectralRegion(5430*u.angstrom, 5442*u.angstrom),
           'OII': specutils.SpectralRegion(4043*u.angstrom, 4052*u.angstrom)
          }

for ax, (line, reg) in zip(axs.ravel(), regions.items()):
    centroid = specutils.analysis.centroid(subspec, reg)
    ax.step(subspec.wavelength, subspec.flux, where='mid')
    ax.axvline(reg.lower.value, c='k', ls='--')
    ax.axvline(reg.upper.value, c='k', ls='--')
    ax.axvline(centroid.value, c='k', ls='-')
    ax.set_xlim((reg.lower-15*u.angstrom).value, (reg.upper+15*u.angstrom).value)
    ax.set_title(line)
    
plt.tight_layout()

As before, the numbers in the "regions" dictionary had to be manually tweaked and the cell re-executed.

But that required a lot of manual plotting.  Even better would be to be able to get regions fit interactively!:

## Interactive version

How to use: Execute the next three cells (until you see a spectrum). 

In the spectrum view, select the region for a spectral line to be identified.  Then hit "Record line".  This should also populate an estimate for a redshift  Select the next line and hit "jump", recording if desired.  Continue until all lines are populated.

In [ ]:
import glue_jupyter as gj
from glue import core as gcore

import ipywidgets

In [ ]:
subspec_data = gcore.Data(wl=subspec.wavelength, flux=subspec.flux, unc=subspec.uncertainty.array)
app = gj.jglue(subspec_data)

In [ ]:
# this cell sets up a "line identification GUI" which is used in the next cell

regions = {}  # populated with spectral regions by frecord

line_selection = ipywidgets.Dropdown(options=bright_lines.keys(), description='Line:')
record_button = ipywidgets.Button(description='Record line')
result_text = ipywidgets.Text(disabled=True, value='No line recorded')


record_box = ipywidgets.HBox([line_selection, record_button, result_text])


z_text = ipywidgets.FloatText(description='z guess:', value='0')
jump_button = ipywidgets.Button(description='Jump to line')

jump_box = ipywidgets.HBox([z_text, jump_button])


def frecord(widget):
    wls_selected = subspec_data['wl'][subspec_data.subsets[0].to_index_list()]
    
    reg = specutils.SpectralRegion(np.min(wls_selected)*subspec.wavelength.unit, 
                                   np.max(wls_selected)*subspec.wavelength.unit)
    result_text.value = 'Recorded {}: {} to {}'.format(line_selection.value, reg.lower, reg.upper)
    regions[line_selection.value] = reg
    
    z_text.value = specutils.analysis.centroid(subspec, reg)/bright_lines[line_selection.value]-1
record_button.on_click(frecord)


_jumplot = [None]
def fjump(widget):
    specplot = _jumplot[0]
    if specplot is None:
        return
    rest_linecen = bright_lines[line_selection.value]
    linecen = rest_linecen*(1+z_text.value)
    specplot.state.x_min = linecen.value + 50
    specplot.state.x_max = linecen.value - 50
jump_button.on_click(fjump)

In [ ]:
specplot = app.scatter2d('wl', 'flux')
_jumplot[0] = specplot

# note that the line below *might* not work on some versions of glue-jupyter. If so, either manually do the brushing, or use this: specplot.button_action.value = 'brush x' 
specplot.widget_button_interact.value = specplot.interact_brush_x
specplot.state.x_min = 7050
specplot.state.x_max = 7250

ipywidgets.VBox([record_box, jump_box])

Improvements desired:

* Hide the state panel and the  - it's a distraction.
* Make the spectrum a "line"
* Show the uncertainty as a fainter line

# Estimate redshift

Lets display the results for all the lines in a table:

In [ ]:
tab = table.QTable()
tab['name'] = table.Column(dtype='S10')
tab['centroid'] = table.Column(dtype=float, unit=u.angstrom)
tab['rest'] = table.Column(dtype=float, unit=u.angstrom)
for name in bright_lines:
    centroid = specutils.analysis.centroid(subspec, regions[name])
    tab.add_row([name, centroid, bright_lines[name]]) 
    
tab['z'] = (tab['centroid']/tab['rest']) - 1
tab

And then estimate the redshift from all of these combined:

In [ ]:
z_mean = np.mean(tab['z'])
z_mean

In [ ]:
z_std = np.std(tab['z'])
z_std

In [ ]:
percent_uncertainty = (z_std/z_mean).to(u.percent)
percent_uncertainty